In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# path = r'./data/raw/'
# all_files = glob.glob(os.path.join(path, "*.csv"))
# df_from_each_file = (pd.read_csv(f) for f in all_files)
# pd.concat(df_from_each_file, ignore_index=True).to_csv('./data/merged.csv', index=False)

In [3]:
df = pd.read_csv('./data/merged.csv')

In [4]:
df.columns

Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'created_at', 'creator', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'friends', 'fx_rate', 'goal', 'id',
       'is_backing', 'is_starrable', 'is_starred', 'launched_at', 'location',
       'name', 'permissions', 'photo', 'pledged', 'profile', 'slug',
       'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'urls', 'usd_pledged', 'usd_type'],
      dtype='object')

In [5]:
df.iloc[8300, 0:]

backers_count                                                               0
blurb                       Relationship Podcast based on our funny and in...
category                    {"id":239,"name":"Radio & Podcasts","slug":"pu...
converted_pledged_amount                                                    0
country                                                                    US
created_at                                                         1395262323
creator                     {"id":1989638862,"name":"Curtis Rambo","is_reg...
currency                                                                  USD
currency_symbol                                                             $
currency_trailing_code                                                   True
current_currency                                                          USD
deadline                                                           1398442191
disable_communication                                           

In [6]:
remove = ['id', 'permissions', 'photo', 'disable_communication', 'converted_pledged_amount', 'pledged', 'currency', 'slug',
       'source_url',  'creator', 'friends', 'profile', 'urls', 'is_backing', 'is_starrable', 'is_starred', 'usd_pledged', 'created_at',
       'backers_count', 'usd_type', 'static_usd_rate','state_changed_at', 'currency_symbol','currency_trailing_code', 'current_currency', 
       'spotlight','staff_pick', 'fx_rate']

In [7]:
df.drop(remove, axis=1, inplace=True)

In [8]:
categories = df['category'].apply(json.loads)

df['category_slug'] = categories.apply(lambda x: x['slug'])

In [9]:
df['main_category'] = df['category_slug'].apply(lambda x: x.split('/'))
df['sub_category'] = df['main_category'].apply(lambda x: x[1] if len(x) > 1 else x[0])
df['main_category'] = df['main_category'].apply(lambda x: x[0])

In [10]:
df['main_category'].value_counts()  

main_category
film & video    27566
music           27237
technology      21100
art             20474
publishing      20123
food            16148
games           13947
fashion         12049
comics           8803
design           8776
photography      8118
crafts           7196
theater          6862
journalism       5815
dance            4043
Name: count, dtype: int64

In [11]:
df['sub_category'].value_counts()

sub_category
web               4569
product design    4257
tabletop games    4054
accessories       3562
comic books       3457
                  ... 
quilts             109
games               95
letterpress         62
chiptune            44
taxidermy           18
Name: count, Length: 159, dtype: int64

In [12]:
df.drop(['category'], axis=1, inplace=True)

In [13]:
# fill nan with 'unknown' for location column
df['location'].fillna('unknown', inplace=True)

In [14]:
def extract_json(df, col, key):
    location_type = []
    for i in range(len(df)):
        if df[col][i] == df[col][i]:
            try:
                location_type.append(json.loads(df[col][i])[key])
            except:
                location_type.append(np.nan)
        else:
            location_type.append(np.nan)
    return location_type


In [15]:
df['location_type'] = extract_json(df, 'location', 'type')

In [16]:
df['location_type'].value_counts()

location_type
Town             193030
County             7831
Suburb             5162
LocalAdmin         1221
Zip                 496
Island              243
Country              27
Miscellaneous        16
Estate                5
Name: count, dtype: int64

In [17]:
# convert date columns to datetime iso format 
df['deadline'] = pd.to_datetime(df['deadline'], unit='s')
df['launched_at'] = pd.to_datetime(df['launched_at'], unit='s')

In [18]:
# create new column for campaign duration
df.loc[:, 'duration'] = df['deadline'] - df['launched_at']

In [19]:
# change campain duration to integer
df['duration'] = df['duration'].apply(lambda x: x.days)

In [20]:
df['duration'].head()

0    49
1    30
2    30
3    59
4    30
Name: duration, dtype: int64

In [21]:
df.loc[:, 'deadline_month'] = df['deadline'].dt.month
df.loc[:, 'deadline_day'] = df['deadline'].dt.day

df.loc[:, 'launched_at_month'] = df['launched_at'].dt.month
df.loc[:, 'launched_at_day'] = df['launched_at'].dt.day

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208257 entries, 0 to 208256
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   blurb              208249 non-null  object        
 1   country            208257 non-null  object        
 2   deadline           208257 non-null  datetime64[ns]
 3   goal               208257 non-null  float64       
 4   launched_at        208257 non-null  datetime64[ns]
 5   location           208257 non-null  object        
 6   name               208257 non-null  object        
 7   state              208257 non-null  object        
 8   category_slug      208257 non-null  object        
 9   main_category      208257 non-null  object        
 10  sub_category       208257 non-null  object        
 11  location_type      208031 non-null  object        
 12  duration           208257 non-null  int64         
 13  deadline_month     208257 non-null  int32   

In [23]:
# check for null values
df.isnull().sum()

blurb                  8
country                0
deadline               0
goal                   0
launched_at            0
location               0
name                   0
state                  0
category_slug          0
main_category          0
sub_category           0
location_type        226
duration               0
deadline_month         0
deadline_day           0
launched_at_month      0
launched_at_day        0
dtype: int64

In [24]:
# fill null values with 'unknown' for location_type column
df['location_type'].fillna('unknown', inplace=True)

In [25]:
# check for duplicates
df.duplicated().sum()
# remove duplicates
df.drop_duplicates(inplace=True)

In [26]:
df.head()

,blurb,country,deadline,goal,launched_at,location,name,state,category_slug,main_category,sub_category,location_type,duration,deadline_month,deadline_day,launched_at_month,launched_at_day
0,Babalus Shoes,US,2019-03-14 05:02:55,28000.0,2019-01-23 06:02:55,"{""id"":2462429,""name"":""Novato"",""slug"":""novato-c...",Babalus Children's Shoes,live,fashion/footwear,fashion,footwear,Town,49,3,14,1,23
1,A colorful Dia de los Muertos themed oracle de...,US,2017-09-09 17:00:59,1000.0,2017-08-10 17:00:59,"{""id"":2400549,""name"":""Euless"",""slug"":""euless-t...",The Ofrenda Oracle Deck,successful,games/playing cards,games,playing cards,Town,30,9,9,8,10
2,"Electra's long awaited, eclectic Debut Pop/Roc...",US,2013-06-12 05:03:15,15000.0,2013-05-13 05:03:15,"{""id"":2423474,""name"":""Hollywood"",""slug"":""holly...","Record Electra's Debut Album (Pop, Rock, Class...",successful,music/rock,music,rock,Suburb,30,6,12,5,13
3,The Mist of Tribunal is a turn-based card game...,GB,2017-03-13 17:22:56,10000.0,2017-01-12 18:22:56,"{""id"":475457,""name"":""Kaunas"",""slug"":""kaunas-ka...",The Mist of Tribunal - A Card Game,failed,games/playing cards,games,playing cards,Town,59,3,13,1,12
4,"Livng with a brain impairment, what its like t...",US,2013-01-09 20:32:07,2800.0,2012-12-10 20:32:07,"{""id"":2507703,""name"":""Traverse City"",""slug"":""t...",Help change the face of Brain Impairment,successful,publishing/nonfiction,publishing,nonfiction,Town,30,1,9,12,10


In [27]:
# drop location
df.drop(['location', 'category_slug', 'deadline', 'launched_at'], axis=1, inplace=True)

In [28]:
df = df[df['state'] != 'live']

In [29]:
df['state'].value_counts()

state
successful    94399
failed        73923
canceled       8581
suspended       619
Name: count, dtype: int64

In [30]:
# save the dataframe as a csv file
df.to_csv('data.csv',index=False)